In [ ]:
import os

main_dir = os.getcwd()
hd_dir = os.path.join(main_dir, 'HD_3.0')
examples_dir = os.path.join(hd_dir, '03_Examples')
case1_dir = os.path.join(examples_dir, 'case1')
export_dir = os.path.join(main_dir, 'results')

In [ ]:
def host_export(host_path, export_dir):
    """
    Convert host structure file to xyz format.
    """
    host_file = open(host_path, 'r')
    host_lines = host_file.readlines()
    host_file.close()
    num_atoms = host_lines[1].split()[0]
    
    xyz_lines = ''
    for line in host_lines[2:2+int(num_atoms)]:
        name = line.split()[0]
        x, y, z = line.split()[2], line.split()[3], line.split()[4]
        xyz_lines += name + '\t' + x + '\t' + y + '\t' + z + '\n'
        
    host_name = os.path.basename(host_path)     
    xyz_path = os.path.join(export_dir, host_name + '.xyz')
    xyz_file = open(xyz_path, 'w')
    xyz_file.write(num_atoms + '\n')
    xyz_file.write(host_name + '\n')
    xyz_file.write(xyz_lines)
    xyz_file.close()
    
def hdo_export(hdo_path, export_dir, num=5):
    """
    Export a given number of structures with lowest RMSD values from hdo.out file. 
    """
    hdo_file = open(hdo_path, 'r')
    hdo_lines = hdo_file.readlines()
    hdo_file.close()

    num_structures = 0
    hdo_structures = {'rmsd': [], 'energy': [], 'num_atoms': [], 'start': [], 'end': []}
    for line_index, line in enumerate(hdo_lines):
        if line.replace(' ', '').strip('\n').isdigit():
            hdo_structures['num_atoms'].append(int(line.replace(' ', '').strip('\n')))    
        if 'RMSD' in line:
            hdo_structures['rmsd'].append(float(line.split(',')[0].split('=')[-1]))
            hdo_structures['energy'].append(float(line.split(',')[2].split('_')[1]))
            hdo_structures['start'].append(line_index)
            num_structures += 1 

    start_line = 2
    for structure in range(num):
        xyz_lines = str(hdo_structures['num_atoms'][structure]) + '\n'
        xyz_lines += str(structure+1) + '_' + str(hdo_structures['rmsd'][structure]) + '\n'

        end_line = start_line + hdo_structures['num_atoms'][structure] 
        for line in hdo_lines[start_line:end_line]:
            name, x, y, z = line.split()[0], line.split()[1], line.split()[2], line.split()[3]
            xyz_lines += name + '\t' + x + '\t' + y + '\t' + z + '\n'

        xyz_path = os.path.join(export_dir, str(structure+1) + '.xyz')
        xyz_file = open(xyz_path, 'w')
        xyz_file.write(xyz_lines)
        xyz_file.close()

        start_line += hdo_structures['num_atoms'][structure] + 2

In [ ]:
hosta_dir = os.path.join(case1_dir, 'hosta')

host_export(hosta_dir, export_dir)

In [ ]:
hdo_path = os.path.join(case1_dir, 'out_1.hdo')

hdo_export(hdo_path, export_dir, num=15)